# Install and import required library

In [1]:
from IPython.display import clear_output
if 'google.colab' in str(get_ipython()):
  !pip install talib-binary
  !pip install yfinance
  !pip install yahoo-fin
  !git clone https://github.com/PeemapatW/backtesting.py_FL.git
  %cd /content/backtesting.py_FL/backtesting
else:
  %cd C:/Users/peema/Documents/GitHub/backtesting.py_FL/backtesting
clear_output()

In [2]:
import json
import requests
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import yfinance
from tqdm import tqdm
from backtesting import Backtest, Strategy
from lib import crossover, cross
import talib as ta
from strategy import *
from util import *
from ipywidgets import widgets

C:\Users\peema\Documents\GitHub\backtesting.py_FL\backtesting\_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

# Backtesting Trading Strategy

In [9]:
#@title Backtesting for MA Cross 2 Line { run: "auto", form-width: "50%" }
strategy = SMA_Cross #@param [ "SMA_Cross","EMA_Cross", "DEMA_Cross","TEMA_Cross","TRIMA_Cross"] {type:"raw"}
ticker = "BTC-USD" #@param ["BTC-USD", "BNB-USD"] {allow-input: true}
start_date = "2010-01-01" #@param {type:"date"}
end_date = "2022-08-24" #@param {type:"date"}
param_fastperiod = 5 #@param {type:"slider", min:0, max:100, step:1}
param_slowperiod = 22 #@param {type:"slider", min:0, max:100, step:1}
param = {"param_fastperiod":param_fastperiod,"param_slowperiod":param_slowperiod}
output1 = widgets.Output()
output2 = widgets.Output()
if param_fastperiod >= param_slowperiod:
  raise ValueError("fastperiod must less than slowperiod")
else:
    bt, output = Test_Strategy(ticker,strategy,param=param,start_date=start_date,end_date=end_date,plot=False)
    bt.plot()
    output_df = pd.DataFrame(output).T.iloc[:,:-3]
    display(output_df.iloc[:,:20])
    display(output_df.iloc[:,20:])

,Start,End,Duration,Exposure Time [%],Equity Final [$],Equity Peak [$],Return [%],Buy & Hold Return [%],DCA Return [%],Return (Ann.) [%],Volatility (Ann.) [%],Sharpe Ratio,Sortino Ratio,Calmar Ratio,Max. Drawdown [%],Avg. Drawdown [%],Max. Drawdown Duration,Avg. Drawdown Duration,# Trades,Win Rate [%]
0,2014-09-17,2022-08-24,2898 days,99.03415,54230829.356737,99553838.909944,5323.082936,4578.204296,1906.08647,65.32979,123.763545,0.52786,1.502224,1.059953,-61.63461,-13.452932,593 days,58 days,144,36.111111


,p-value for Win Rate > 50%,Win Rate > 50%,mean P&L,t_score for mean P&L > 0,p-value for mean P&L > 0,mean P&L > 0,Best Trade [%],Worst Trade [%],Avg. Trade [%],Max. Trade Duration,Avg. Trade Duration,Profit Factor,Expectancy [%],SQN
0,0.999707,False,369658.5372,0.911946,0.181666,False,122.077493,-23.393531,2.812151,100 days,20 days,2.07518,4.504903,0.911946


In [73]:
SET50_name = [ticker+'.BK' for ticker in ['AOT','PTT','PTTEP','ADVANC','GULF','CPALL','SCC','BDMS','KBANK','SCB','OR','EA','CPN','IVL','BBL','SCGP','CPF','INTUCH','KTB','CRC','PTTGC','GPSC',
 'MINT','HMPRO','AWC','TRUE','KTC','BH','BEM','TTB','BTS','JMT','CBG','DTAC','TOP','OSP','LH','EGCO','BGRIM','MTC','GLOBAL','BANPU','TU','TIDLOR',
 'JMART','KCE','TISCO','BLA','SAWAD','IRPC']]
TopCrypto_name = [ticker+'-USD' for ticker in ['BTC','ETH','BNB','XRP','ADA','SOL','DOGE','HEX','DOT','SHIB','WTRX','MATIC','AVAX','TRX','STETH','WBTC','ETC','UNI1','LEO','YOUC','LTC','FTT','LINK','CRO','NEAR','ATOM','XMR','XLM','BCH','FLOW','ALGO','VET','FIL','ICP',
                  'APE3','MANA','EOS','SAND','HBAR','XTZ','TONCOIN','QNT','WBNB','EGLD','THETA','AAVE','CHZ','AXS','OKB','BSV']]
Asset_name = SET50_name + TopCrypto_name

start_date_widget = widgets.DatePicker(value=datetime(2010,1,1),description='Pick a Date',disabled=False)
end_date_widget = widgets.DatePicker(value=datetime.now(),description='Pick a Date',disabled=False)
strategy_widget = widgets.Select(
    options=[ "SMA_Cross","EMA_Cross", "DEMA_Cross","TEMA_Cross","TRIMA_Cross"],
    value='SMA_Cross',
    # rows=10,
    description='Strategy:',
    disabled=False
)
ticker_widget = widgets.Combobox(
    value='',
    placeholder='Asset Name',
    options=Asset_name,
    description='Asset Name:',
    ensure_option=False,
    disabled=False
)
param_widget = widgets.IntRangeSlider(
    value=[5, 10],
    min=1,
    max=100,
    step=1,
    description='Test:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d',
)

In [75]:
widgets.VBox([widgets.HBox([start_date_widget,end_date_widget]),strategy_widget,ticker_widget,param_widget])

In [53]:
button = widgets.Button(
    description='Click me',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='check' # (FontAwesome names without the `fa-` prefix)
)
button

Button(description='Click me', icon='check', style=ButtonStyle(), tooltip='Click me')

In [69]:
display(button)
button.on_click(run)

Button(description='Click me', icon='check', style=ButtonStyle(), tooltip='Click me')

In [48]:
button = widgets.Button(description="Click Me!")
display(button)

def fun(a):
    print(a)

def on_button_clicked(b):
    fun('a')

button.on_click(on_button_clicked)

Button(description='Click Me!', style=ButtonStyle())

In [66]:
#@title Backtesting for MA Cross 2 Line { run: "auto", form-width: "50%" }
def run(trigger):
  strategy = strategy_widget.value
  ticker = ticker_widget.value
  start_date = start_date_widget.value
  end_date = end_date_widget.value
  param_fastperiod,param_slowperiod = param_widget.value
  param = {"param_fastperiod":param_fastperiod,"param_slowperiod":param_slowperiod}
  output1 = widgets.Output()
  output2 = widgets.Output()
  if param_fastperiod >= param_slowperiod:
    raise ValueError("fastperiod must less than slowperiod")
  else:
      bt, output = Test_Strategy(ticker,eval(strategy),param=param,start_date=start_date,end_date=end_date,plot=False)
      bt.plot()
      output_df = pd.DataFrame(output).T.iloc[:,:-3]
      display(output_df.iloc[:,:20])
      display(output_df.iloc[:,20:])

In [67]:
run('a')

,Start,End,Duration,Exposure Time [%],Equity Final [$],Equity Peak [$],Return [%],Buy & Hold Return [%],DCA Return [%],Return (Ann.) [%],Volatility (Ann.) [%],Sharpe Ratio,Sortino Ratio,Calmar Ratio,Max. Drawdown [%],Avg. Drawdown [%],Max. Drawdown Duration,Avg. Drawdown Duration,# Trades,Win Rate [%]
0,2014-09-17,2022-08-25,2899 days,97.862069,182467052.018327,227820789.712183,18146.705202,4597.254923,1914.255674,92.573868,139.195952,0.665061,2.172188,1.336126,-69.285307,-11.986987,315 days,38 days,44,54.545455


,p-value for Win Rate > 50%,Win Rate > 50%,mean P&L,t_score for mean P&L > 0,p-value for mean P&L > 0,mean P&L > 0,Best Trade [%],Worst Trade [%],Avg. Trade [%],Max. Trade Duration,Avg. Trade Duration,Profit Factor,Expectancy [%],SQN
0,0.325794,False,4124251.182235,1.553881,0.063771,False,353.192937,-47.162821,12.56468,214 days,65 days,5.557359,24.387097,1.553881
